In [1]:
import jax
from jax.config import config
config.update("jax_enable_x64", True)

In [2]:
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import moscot
from moscot.problems.time import TemporalProblem
import moscot.plotting as mpl
import pandas as pd
import os

sc.set_figure_params(scanpy=True, dpi=80, dpi_save=200)

import mplscience

mplscience.available_styles()
mplscience.set_style(reset_current=True)
plt.rcParams['legend.scatterpoints'] = 1 

['default', 'despine']


In [3]:
output_dir = "/lustre/groups/ml01/workspace/moscot_paper/pancreas/driver_features"

In [4]:
chromvar_features = pd.read_csv("/lustre/groups/ml01/workspace/moscot_paper/pancreas/cisBP_chromvar_annotations_reduced.csv")

In [5]:
chromvar_features.set_index("Unnamed: 0", inplace=True)

In [6]:
chromvar_features = chromvar_features.T

In [7]:
chromvar_features["old_index"] = chromvar_features.index

In [8]:
def rename_index(x):
    split = x["old_index"].split("_")
    if split[0] == "E14-5":
        return split[1]+"-0"
    if split[0] == "E15-5":
        return split[1]+"-1"
    raise ValueError("Pattern does not match: ", split)

In [9]:
chromvar_features["index_adapted"] = chromvar_features.apply(rename_index ,axis=1)

In [10]:
chromvar_features.set_index("index_adapted", inplace=True)

In [11]:
motif2TF = pd.read_csv("/lustre/groups/ml01/workspace/moscot_paper/pancreas/motif2TF.csv")

In [12]:
motif2TF["TF_name"] = motif2TF.apply(lambda x: x["TF_Name"].replace("[", "").replace("]", "").replace("'", "").split(), axis=1)

In [13]:
del motif2TF["TF_Name"]

In [14]:
del chromvar_features["old_index"]

In [15]:
chromvar_features.head()

Unnamed: 0,new motif,new motif.1,new motif.2,new motif.3,new motif.4,new motif.5,new motif.6,new motif.7,new motif.8,new motif.9,...,new motif.6004,new motif.6005,new motif.6006,new motif.6007,new motif.6008,new motif.6009,new motif.6010,new motif.6011,new motif.6012,new motif.6013
index_adapted,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAACAGCCT-1-0,1.449150,-1.548024,0.127186,-0.401432,5.779336,-0.047665,-0.104796,0.082138,-0.065305,-0.163256,...,-1.134490,0.531467,-0.404171,-2.334780,0.179397,-0.388121,0.772678,0.357471,0.428473,0.772678
AAACAGCCAACCCTCC-1-0,-0.986286,3.096357,2.472101,0.594340,5.514991,1.100158,-0.874710,-0.670196,0.534449,-0.349137,...,-1.157848,-1.993996,0.308834,-1.702847,-1.646581,1.172381,0.286935,-0.433321,2.500077,0.286935
AAACAGCCACCTGTAA-1-0,2.217122,-0.694262,-1.776413,-0.556984,-0.999821,-1.439918,-0.359144,0.231731,-1.116241,-0.331855,...,1.615487,-0.255339,-1.269120,2.926865,0.709345,0.389420,-1.620937,-1.635631,-0.787535,-1.620937
AAACAGCCACTAAGCC-1-0,1.043075,1.337077,-0.491503,1.104463,5.234869,-0.295337,0.489305,-1.783421,-1.110079,-0.149686,...,-0.439479,-0.452744,-1.027292,-2.424418,-0.645681,-0.100782,2.136637,0.185030,0.231816,2.136637
AAACAGCCAGGATAAC-1-0,0.537357,-0.343018,-1.047688,-1.379141,0.126564,-0.931787,-0.365652,-1.572852,-1.259440,-1.114664,...,0.704474,-0.813751,-1.530833,-0.959602,-1.077125,-1.498234,1.009304,1.857225,1.016590,1.009304


In [16]:
motifs = motif2TF["Motif_ID"].values
rename_cols = {col: motifs[i] for i, col in enumerate(chromvar_features.columns)}

In [17]:
chromvar_features.rename(columns=rename_cols, inplace=True)

In [18]:
chromvar_features

Unnamed: 0,M00008_2.00,M00044_2.00,M00056_2.00,M00060_2.00,M00066_2.00,M00070_2.00,M00111_2.00,M00112_2.00,M00113_2.00,M00114_2.00,...,M11350_2.00,M11361_2.00,M11367_2.00,M11368_2.00,M11371_2.00,M11384_2.00,M11388_2.00,M11389_2.00,M11390_2.00,M11491_2.00
index_adapted,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAACAGCCT-1-0,1.449150,-1.548024,0.127186,-0.401432,5.779336,-0.047665,-0.104796,0.082138,-0.065305,-0.163256,...,-1.134490,0.531467,-0.404171,-2.334780,0.179397,-0.388121,0.772678,0.357471,0.428473,0.772678
AAACAGCCAACCCTCC-1-0,-0.986286,3.096357,2.472101,0.594340,5.514991,1.100158,-0.874710,-0.670196,0.534449,-0.349137,...,-1.157848,-1.993996,0.308834,-1.702847,-1.646581,1.172381,0.286935,-0.433321,2.500077,0.286935
AAACAGCCACCTGTAA-1-0,2.217122,-0.694262,-1.776413,-0.556984,-0.999821,-1.439918,-0.359144,0.231731,-1.116241,-0.331855,...,1.615487,-0.255339,-1.269120,2.926865,0.709345,0.389420,-1.620937,-1.635631,-0.787535,-1.620937
AAACAGCCACTAAGCC-1-0,1.043075,1.337077,-0.491503,1.104463,5.234869,-0.295337,0.489305,-1.783421,-1.110079,-0.149686,...,-0.439479,-0.452744,-1.027292,-2.424418,-0.645681,-0.100782,2.136637,0.185030,0.231816,2.136637
AAACAGCCAGGATAAC-1-0,0.537357,-0.343018,-1.047688,-1.379141,0.126564,-0.931787,-0.365652,-1.572852,-1.259440,-1.114664,...,0.704474,-0.813751,-1.530833,-0.959602,-1.077125,-1.498234,1.009304,1.857225,1.016590,1.009304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCTTGTTCG-1-1,-1.843325,-2.941034,-0.081693,3.176968,-2.878943,0.691116,-1.438785,-1.751296,-1.575429,-0.198032,...,-0.177601,0.614194,0.973428,3.402909,2.875351,0.054628,-1.024672,0.414746,-0.990464,-1.024672
TTTGTTGGTATTGAGT-1-1,0.339756,-0.607060,1.182701,-1.404739,-0.471032,-0.241393,0.664326,1.204081,0.279564,1.113933,...,0.392067,-0.956582,-1.057869,0.607653,0.115554,1.749512,-0.751267,1.864898,-2.003164,-0.751267
TTTGTTGGTCCTAAGA-1-1,0.715763,-1.945611,0.135289,1.317406,-2.127201,1.142669,0.559061,0.254880,-0.020150,0.280686,...,-1.672752,0.177487,1.239300,0.052760,1.178176,-0.367450,-1.006079,1.428689,0.282479,-1.006079


In [19]:
chromvar_features.to_csv("/lustre/groups/ml01/workspace/moscot_paper/pancreas/cisBP_chromvar_features_processed.csv")

In [20]:
motif2TF.head()

,Motif_ID,direct_TF,TF_name
0,M00008_2.00,NaN,"[Setbp1, Ahctf1, Hmga1, Hmga2, Phf21a, Hmga1-rs1]"
1,M00044_2.00,NaN,[Foxp3]
2,M00056_2.00,NaN,"[A1JVI6_MOUSE, Sebox, Phox2a, Phox2b, Alx3, Vs..."
3,M00060_2.00,NaN,"[Hsf2, Hsf4]"
4,M00066_2.00,NaN,"[Rfx4, Rfx2, Rfx1, Rfx3]"


In [21]:
motif2TF_dict = pd.Series(motif2TF["TF_name"].values,index=motif2TF["Motif_ID"].values).to_dict()
motif2TF_direct_dict = pd.Series(motif2TF["direct_TF"].values,index=motif2TF["Motif_ID"].values).to_dict()

In [22]:
tp0 = TemporalProblem.load("/lustre/groups/ml01/workspace/moscot_paper/pancreas/TemporalProblem.pkl")

In [23]:
tp0.adata.obs = tp0.adata.obs.merge(chromvar_features, left_index=True, right_index=True)

In [24]:
order_celltypes = ['Ngn3 low', 'Ngn3 high cycling', 'Ngn3 high', 'Eps. progenitors', 'Fev+', 'Fev+ Alpha', 
                   'Fev+ Beta', 'Fev+ Delta', 'Alpha', 'Beta', 'Delta', 'Epsilon']

In [25]:
color_dict = { 'Prlf. Ductal': '#f7b6d2', 'Ductal': '#d62728', 'Imm. Acinar': '#ffbb78', 'Mat. Acinar': '#98df8a',
    'Alpha': '#1f77b4', 'Beta': '#ff7f0e', 'Delta': '#279e68', 'Eps. progenitors': '#aa40fc', 'Epsilon': '#8c564b',
       'Fev+': '#e377c2', 'Fev+ Alpha': '#b5bd61', 'Fev+ Beta': '#aec7e8', 'Fev+ Delta': '#aec7e8', 'Fev+ Delta,0': '#ffbb78', 'Fev+ Delta,1':'#98df8a',
        'Fev+ Delta,2': '#17becf', 'Ngn3 high cycling': '#aec7e8',
        'Ngn3 high,0': '#ff9896', 'Ngn3 high,1':'#f0b98d', 'Ngn3 low': '#c5b0d5'
}

In [26]:
tp0.adata.obs["refinement"] = tp0.adata.obs["refinement"].astype("category")
tp0.adata.uns["refinement_colors"] = [color_dict[ct] for ct in tp0.adata.obs["refinement"].cat.categories]

In [27]:
annotation1 = {"celltype": ["Alpha", "Beta", "Epsilon", "Delta", "Fev+ Alpha", "Fev+ Beta", "Fev+ Delta", "Eps. progenitors"]}
annotation2 = {"celltype": ["Delta", "Fev+ Delta", "Eps. progenitors", "Ngn3 high"]}
annotation3 = {"celltype": ["Epsilon", "Fev+ Delta", "Eps. progenitors", "Ngn3 high", "Alpha"]}

In [28]:
chromvar_features = [col for col in tp0.adata.obs.columns if col.startswith("M")]

In [29]:
len(chromvar_features)

6014

# Delta

In [30]:
tp0.pull(14.5, 15.5, data="celltype", subset="Delta", key_added="Delta_pull")

In [31]:
delta_drivers = tp0.compute_feature_correlation(obs_key="Delta_pull", features=chromvar_features, annotation=annotation2)

In [32]:
delta_drivers["TFs"] = delta_drivers.apply(lambda x: motif2TF_dict[x.name], axis=1)
delta_drivers["TF_direct"] = delta_drivers.apply(lambda x: motif2TF_direct_dict[x.name], axis=1)


In [33]:
delta_drivers

,Delta_pull_corr,Delta_pull_pval,Delta_pull_qval,Delta_pull_ci_low,Delta_pull_ci_high,TFs,TF_direct
M09209_2.00,0.411525,4.001885e-183,1.203367e-179,0.386548,0.435898,"[Sebox, Shox2, Isl2, Vsx1, Isl1, Noto]",Isl1
M09128_2.00,0.411525,4.001885e-183,1.203367e-179,0.386548,0.435898,"[Sebox, Shox2, Isl2, Vsx1, Isl1, Noto]",NaN
M05204_2.00,0.355137,1.661457e-132,3.330668e-129,0.328905,0.380821,"[Sebox, Shox2, Isl2, Vsx1, Isl1, Noto]",NaN
M03291_2.00,0.343129,4.544313e-123,6.832375e-120,0.316654,0.369069,"[Pax2, Pax6]",NaN
M00381_2.00,0.339795,1.581686e-120,1.585377e-117,0.313256,0.365805,"[Hoxb6, Hoxa10, Hoxa4, Mnx1, Hoxc8, Hoxc6, Hox...",Hoxb6
...,...,...,...,...,...,...,...
M08735_2.00,-0.336816,2.772099e-118,1.282416e-115,-0.362887,-0.310218,"[Neurod1, Neurod6, Neurod2, Neurod4]",NaN
M04168_2.00,-0.336818,2.762780e-118,1.282416e-115,-0.362889,-0.310220,[Atoh1],NaN
M08758_2.00,-0.337101,1.694695e-118,1.019189e-115,-0.363166,-0.310509,"[Neurog2, Neurog3]",Neurog2
M04174_2.00,-0.337865,4.528548e-119,3.026076e-116,-0.363914,-0.311287,"[Olig2, Olig3]",NaN


In [34]:
delta_drivers.to_csv(os.path.join(output_dir, "driver_delta_motifs.csv"))

# Beta

In [35]:
tp0.pull(14.5, 15.5, data="celltype", subset="Beta", key_added="Beta_pull")

In [36]:
beta_drivers = tp0.compute_feature_correlation(obs_key="Beta_pull", features=chromvar_features,  annotation=annotation1)

In [37]:
beta_drivers["TFs"] = beta_drivers.apply(lambda x: motif2TF_dict[x.name], axis=1)
beta_drivers["TF_direct"] = beta_drivers.apply(lambda x: motif2TF_direct_dict[x.name], axis=1)

In [38]:
beta_drivers

,Beta_pull_corr,Beta_pull_pval,Beta_pull_qval,Beta_pull_ci_low,Beta_pull_ci_high,TFs,TF_direct
M08835_2.00,0.564177,1.202505e-269,3.615933e-266,0.539354,0.588020,"[Mafk, Mafa, Mafg, Maf, Mafb]",Mafg
M08809_2.00,0.564177,1.202505e-269,3.615933e-266,0.539354,0.588020,"[Mafk, Mafa, Mafg, Maf, Mafb]",NaN
M08834_2.00,0.562532,1.241658e-267,2.489111e-264,0.537643,0.586441,[Maff],Maff
M08807_2.00,0.553100,2.475111e-256,3.721330e-253,0.527837,0.577385,"[Mafk, Mafa, Mafg, Maf, Mafb]",NaN
M08810_2.00,0.551909,6.166693e-255,7.417298e-252,0.526600,0.576242,"[Mafk, Mafa, Mafg, Maf, Mafb]",NaN
...,...,...,...,...,...,...,...
M04837_2.00,-0.360061,3.588118e-95,2.500457e-94,-0.390721,-0.328603,"[Foxa1, Foxa2, Foxa3, Foxb2, Foxb1]",NaN
M04832_2.00,-0.364308,1.320859e-97,9.593774e-97,-0.394855,-0.332957,"[Foxa1, Foxa2, Foxa3, Foxb1, Foxs1]",NaN
M03743_2.00,-0.366171,1.098121e-98,8.034184e-98,-0.396668,-0.334867,"[Foxa1, Foxa2, Foxa3, Foxb2, Foxb1]",NaN
M04834_2.00,-0.368802,3.176557e-100,2.384995e-99,-0.399228,-0.337564,"[Foxa1, Foxa2, Foxa3, Foxb1, Foxs1]",NaN


In [39]:
beta_drivers.to_csv(os.path.join(output_dir, "driver_beta_motifs.csv"))

# Alpha

In [40]:
tp0.pull(14.5, 15.5, data="celltype", subset="Alpha", key_added="Alpha_pull")

In [41]:
alpha_drivers = tp0.compute_feature_correlation(obs_key="Alpha_pull", features=chromvar_features, annotation=annotation1)

In [42]:
alpha_drivers["TFs"] = alpha_drivers.apply(lambda x: motif2TF_dict[x.name], axis=1)
alpha_drivers["TF_direct"] = alpha_drivers.apply(lambda x: motif2TF_direct_dict[x.name], axis=1)

In [43]:
alpha_drivers

,Alpha_pull_corr,Alpha_pull_pval,Alpha_pull_qval,Alpha_pull_ci_low,Alpha_pull_ci_high,TFs,TF_direct
M08123_2.00,0.442097,7.440010e-150,4.474422e-146,0.412931,0.470357,"[Foxa1, Foxa2, Foxa3]",NaN
M09543_2.00,0.433667,1.971305e-143,5.927713e-140,0.404243,0.462195,"[Foxa1, Foxa2, Foxa3, Foxb1]",NaN
M01983_2.00,0.432928,7.060805e-143,1.415456e-139,0.403480,0.461479,"[Foxa1, Foxa2, Foxa3, Foxb2, Foxb1]",Foxb1
M07961_2.00,0.430942,2.132313e-141,3.205932e-138,0.401434,0.459555,"[Foxa1, Foxa2, Foxa3, Foxb1]",NaN
M09088_2.00,0.429040,5.441834e-140,6.545438e-137,0.399476,0.457713,"[Foxa1, Foxa2, Foxa3, Foxb2, Foxb1]",NaN
...,...,...,...,...,...,...,...
M04111_2.00,-0.279534,5.118695e-56,2.523265e-54,-0.312101,-0.246311,"[Bhlhe22, Bhlhe23]",NaN
M03269_2.00,-0.280382,2.296721e-56,1.141527e-54,-0.312931,-0.247175,"[A1JVI6_MOUSE, Lhx2, Sebox, Meox1, Dlx3, Mnx1,...",Arx
M04093_2.00,-0.285231,2.223987e-58,1.183633e-56,-0.317677,-0.252117,"[Msc, Tcf21]",NaN
M04110_2.00,-0.286323,7.722034e-59,4.422887e-57,-0.318746,-0.253230,"[Bhlhe22, Bhlhe23]",NaN


In [44]:
alpha_drivers.to_csv(os.path.join(output_dir, "driver_alpha_motifs.csv"))

# Epsilon

In [45]:
tp0.pull(14.5, 15.5, data="celltype", subset="Epsilon", key_added="Epsilon_pull")

In [46]:
eps_drivers = tp0.compute_feature_correlation(obs_key="Epsilon_pull", features=chromvar_features, annotation=annotation3)

In [47]:
eps_drivers["TFs"] = eps_drivers.apply(lambda x: motif2TF_dict[x.name], axis=1)
eps_drivers["TF_direct"] = eps_drivers.apply(lambda x: motif2TF_direct_dict[x.name], axis=1)

In [48]:
eps_drivers

,Epsilon_pull_corr,Epsilon_pull_pval,Epsilon_pull_qval,Epsilon_pull_ci_low,Epsilon_pull_ci_high,TFs,TF_direct
M09128_2.00,0.223125,9.511563e-58,2.860127e-54,0.196582,0.249342,"[Sebox, Shox2, Isl2, Vsx1, Isl1, Noto]",NaN
M09209_2.00,0.223125,9.511563e-58,2.860127e-54,0.196582,0.249342,"[Sebox, Shox2, Isl2, Vsx1, Isl1, Noto]",Isl1
M03030_2.00,0.205078,8.246654e-49,1.653179e-45,0.178333,0.231521,"[Foxa1, Foxa2, Foxa3, Foxb2, Foxb1]",NaN
M09090_2.00,0.201024,6.440260e-47,9.682931e-44,0.174236,0.227516,"[Foxa1, Foxa2, Foxa3, Foxb1]",NaN
M00665_2.00,0.200505,1.117384e-46,1.066556e-43,0.173711,0.227003,"[Foxb2, Foxb1]",NaN
...,...,...,...,...,...,...,...
M09125_2.00,-0.180303,6.896835e-38,7.276766e-36,-0.207028,-0.153310,"[Grhl1, Grhl2]",NaN
M03246_2.00,-0.181095,3.261033e-38,3.502116e-36,-0.207811,-0.154109,"[Hnf1b, Hmbox1, Hnf1a]",NaN
M03071_2.00,-0.186005,2.886703e-40,3.945598e-38,-0.212668,-0.159065,[Grhl1],NaN
M03245_2.00,-0.189414,1.000658e-41,1.880613e-39,-0.216039,-0.162507,"[Hnf1b, Hmbox1, Hnf1a]",NaN


In [49]:
eps_drivers.to_csv(os.path.join(output_dir, "driver_eps_motifs.csv"))

# Eps. progenitors

In [50]:
tp0.pull(14.5, 15.5, data="celltype", subset="Eps. progenitors", key_added="Eps_prog_pull")

In [51]:
eps_prog_drivers = tp0.compute_feature_correlation(obs_key="Eps_prog_pull", features=chromvar_features, annotation=annotation3)

In [52]:
eps_prog_drivers["TFs"] = eps_prog_drivers.apply(lambda x: motif2TF_dict[x.name], axis=1)
eps_prog_drivers["TF_direct"] = eps_prog_drivers.apply(lambda x: motif2TF_direct_dict[x.name], axis=1)

In [53]:
eps_prog_drivers

,Eps_prog_pull_corr,Eps_prog_pull_pval,Eps_prog_pull_qval,Eps_prog_pull_ci_low,Eps_prog_pull_ci_high,TFs,TF_direct
M03479_2.00,0.100682,9.992973e-13,1.717078e-10,0.073125,0.128085,"[Nfix, Nfib, Nfia, Nfic]",NaN
M03480_2.00,0.100017,1.408427e-12,2.352855e-10,0.072457,0.127424,"[Nfix, Nfib, Nfia, Nfic]",NaN
M03478_2.00,0.098992,2.380981e-12,3.768216e-10,0.071427,0.126405,"[Nfix, Nfib, Nfia, Nfic]",NaN
M03475_2.00,0.098481,3.087159e-12,4.760557e-10,0.070914,0.125898,"[Nfix, Nfib, Nfia, Nfic]",NaN
M05747_2.00,0.096162,9.862349e-12,1.446638e-09,0.068585,0.123593,"[Nfix, Nfib, Nfia, Nfic]",NaN
...,...,...,...,...,...,...,...
M02451_2.00,-0.116703,1.284387e-16,1.143052e-13,-0.143998,-0.089232,"[Rfx6, Rfx4, Rfx2, Rfx1, Rfx3, Rfx8]",NaN
M08010_2.00,-0.117534,7.784481e-17,1.143052e-13,-0.144823,-0.090067,"[Rfx5, Rfx7]",NaN
M09365_2.00,-0.117647,7.268939e-17,1.143052e-13,-0.144935,-0.090181,"[Rfx6, Rfx4, Rfx2, Rfx1, Rfx3, Rfx8]",Rfx6
M03457_2.00,-0.118912,3.365426e-17,1.011984e-13,-0.146191,-0.091453,"[Rfx6, Rfx4, Rfx2, Rfx1, Rfx3, Rfx8]",NaN


In [54]:
eps_prog_drivers.to_csv(os.path.join(output_dir, "driver_eps_prog_motifs.csv"))

# Fev+ Delta

In [55]:
tp0.pull(14.5, 15.5, data="celltype", subset="Fev+ Delta", key_added="Fev_delta_pull")

In [56]:
Fev_delta_drivers = tp0.compute_feature_correlation(obs_key="Fev_delta_pull", features=chromvar_features, annotation=annotation3)

In [57]:
Fev_delta_drivers["TFs"] = Fev_delta_drivers.apply(lambda x: motif2TF_dict[x.name], axis=1)
Fev_delta_drivers["TF_direct"] = Fev_delta_drivers.apply(lambda x: motif2TF_direct_dict[x.name], axis=1)

In [58]:
Fev_delta_drivers

,Fev_delta_pull_corr,Fev_delta_pull_pval,Fev_delta_pull_qval,Fev_delta_pull_ci_low,Fev_delta_pull_ci_high,TFs,TF_direct
M09975_2.00,0.164477,1.058360e-31,6.364979e-28,0.137343,0.191364,"[Bach1, Bach2]",NaN
M08067_2.00,0.161420,1.413194e-30,4.249475e-27,0.134260,0.188337,"[Jun, Junb, Jund]",NaN
M07816_2.00,0.152755,1.655168e-27,3.273379e-24,0.125527,0.179754,"[Jun, Junb, Jund]",NaN
M07827_2.00,0.152409,2.177173e-27,3.273379e-24,0.125177,0.179411,"[Fosb, Fos, Fosl1, Fosl2]",NaN
M08073_2.00,0.151335,5.068503e-27,6.096396e-24,0.124095,0.178347,"[Jun, Junb, Jund]",NaN
...,...,...,...,...,...,...,...
M03245_2.00,-0.121386,7.286218e-18,2.980906e-16,-0.148647,-0.093942,"[Hnf1b, Hmbox1, Hnf1a]",NaN
M02916_2.00,-0.122955,2.716844e-18,1.167079e-16,-0.150203,-0.095520,[Zbtb18],NaN
M02917_2.00,-0.123446,1.989310e-18,8.669353e-17,-0.150691,-0.096014,[Zbtb18],NaN
M04585_2.00,-0.128793,6.165476e-20,3.340466e-18,-0.155996,-0.101394,[Zbtb18],NaN


In [59]:
Fev_delta_drivers.to_csv(os.path.join(output_dir, "driver_Fev_delta_motifs.csv"))

In [62]:
tp0.adata.obs.to_csv("/lustre/groups/ml01/workspace/moscot_paper/pancreas/adata_obs_cisBP_chromvar_features.h5ad")

In [60]:
#delta_drivers_genes = set([el for ls in delta_drivers.head(n=50)["TFs"].values for el in ls]).intersection(tp0.adata.var_names)